In [1]:
from google import genai


In [5]:
from dotenv import load_dotenv
envpath = "../.env"
load_dotenv(envpath)
import os
import sys

API=os.getenv("GOOGLEAPI")
client = genai.Client(api_key=API)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or informed decisions.


In [7]:
response.text

'AI learns patterns from data to make predictions or informed decisions.'

In [8]:
result = client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

[ContentEmbedding(
  values=[
    -0.022374554,
    -0.004560777,
    0.013309286,
    -0.0545072,
    -0.02090443,
    <... 3067 more items ...>,
  ]
)]


In [15]:
def geminiresponse(userinput:str):
    client = genai.Client(api_key=API)

    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=userinput
    )
    return response.text


In [16]:
test = geminiresponse("what are you")

In [17]:
print(test)

I am a large language model, trained by Google.

My purpose is to process information and generate human-like text based on the vast amount of data I was trained on. I can understand and generate human-like text, answer questions, provide information, and engage in conversations on a wide range of topics.

I don't have personal experiences, emotions, consciousness, or a physical body. I am a tool designed to assist with information and communication.


### PROMPT ENGINEERING


In [18]:
from pypdf import PdfReader
reader=PdfReader("../UmarKhattab_Resume.pdf")
#Most PDF are 1 page. this is for 1 page pdf for now. Will make it for multiple pages
page = reader.pages[0] 
page_content = page.extract_text()

In [33]:
system_prompt = """
You are a Hiring Mananger.What do you think about this resume.
"""

def build_prompt(uploaded_text=""):
    prompt = system_prompt + "\n\n"
    # for msg in history:
    #     role = msg['role']
    #     prompt += f"{role.capitalize()}: {msg['content']}\n"
    if uploaded_text:
        prompt += f"Uploaded file:\n\"\"\"\n{uploaded_text}\n\"\"\"\n"
    prompt += "Assistant:"
    return prompt

In [34]:
output = geminiresponse(build_prompt(page_content))

In [35]:
print(output)

Okay, looking at Raja Umar Khattab's resume as a Hiring Manager, here's my assessment:

---

**Overall Impression:**

This is a **strong resume for an entry-level or junior Data Science/Machine Learning/Data Engineering role.** The candidate clearly has a passion for the field, a solid foundational understanding, and has gone above and beyond to build practical projects. The quantitative results in the internships are excellent.

---

**Strengths:**

1.  **Excellent Project Portfolio:** This is the absolute stand-out section. The projects are highly relevant to modern data/AI roles (LLMs, RAG, FastAPI, Docker, CI/CD, NLP, Web Scraping, Vector Databases). They demonstrate:
    *   **Proactive Learning:** Building these complex projects shows initiative beyond coursework.
    *   **Practical Application:** He's not just listing skills; he's showing *how* he applies them.
    *   **Modern Tech Stack:** Using things like Pinecone, Claude, FastAPI, HuggingFace LLMs, Streamlit, and PySpark i